# 동적 페이지 크롤링하기(2)

동적 페이지로만 짜여진 페이지는 전용 페이지 url을 찾아야한다.   
예시로 맛집 정보 공유 사이트인 블루리본서베이 사이트 크롤링을 해보도록 하자.

disable JS 를 통해 JS임을 확인했다.  
### 자바스크립트의 URL을 찾아야 한다.  



### JavaScript URL 찾기
### Network탭에 들어가면 자바스크립트 페이지의 URL을 알아낼 수 있다.
1. network 탭으로 이동(clear - 새로고침)
2. XHR, JS 의 후보들을 하나씩 확인(preview)
    - 페이지(왼쪽)과 preview(오른쪽, json) 대조
3. 해당 요소를 찾았다면, 
   headers(preview 옆)로 이동 -> General - Requests URL


그렇다. 그 url이 우리가 원하던 URL이다.

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd 
import json

In [3]:
url = 'https://www.bluer.co.kr/api/v1/restaurants?page=0&size=30&query=&foodType=&foodTypeDetail=&feature=&location=&locationDetail=&area=&areaDetail=&priceRange=&ribbonType=&recommended=false&isSearchName=false&tabMode=single&searchMode=ribbonType&zone1=&zone2=&zone2Lat=&zone2Lng='

resp = requests.get(url)
resp

<Response [200]>

In [ ]:
# soup = BeautifulSoup(resp.content, 'lxml')
# soup.select()

JSON은 계층적인 딕셔너리의 문자열 형태다.
- JSON은 딕셔너리와 같은 구조다.
- JSON은 계층적인 구조다. 
- JSON은 문자열이다.

### JSON에서 원하는 정보 뽑아내기
soup.select를 쓸 수 없다!! HTML이 아니라 JSON이라서..

page -> JSON

HTML  vs  JSON

둘다 page 데이터 

형식의 차이가 있다.

HTML : tag의 계층구조 -> BeautifulSoup, select(수식어)

JSON : dic 의 계층구조 -> 필요 X, dictionary, key


해결책!  
1) JSON을 딕셔너리로 변환한다.  
2) 딕셔너리에서 key값을 이용해 원하는 정보를 뽑아낸다.


In [4]:
import json

# 1) JSON을 딕셔너리로 변환
result = json.loads(resp.content) # -> JSON
result

{'_embedded': {'restaurants': [{'createdDate': 1436236247000,
    'id': 28167,
    'headerInfo': {'nameKR': '권숙수',
     'nameEN': 'KOREAN DINING KWONSOOKSOO',
     'nameCN': '',
     'nickname': '',
     'year': '2023',
     'bookYear': '2023',
     'ribbonType': 'RIBBON_THREE',
     'ribbonTypeByOrdinal': 0},
    'defaultInfo': {'chefName': '권우중',
     'website': 'http://kwonsooksoo.com',
     'websiteInstagram': 'https://www.instagram.com/kwonsooksoo/',
     'websiteFacebook': '',
     'phone': '02-542-6268 ',
     'openHours': '',
     'closeHours': '',
     'openHoursWeekend': '',
     'closeHoursWeekend': '',
     'dayOff': '일, 월요일, 명절 휴무',
     'bcPoint': True},
    'statusInfo': {'storeType': 'NOT',
     'parking': '발레 파킹',
     'creditCard': 'y',
     'visit': '청담사거리에서 학동사거리 방향으로 직진. 순수도산점에서 우회전하면 우측',
     'menu': '점심숙수상(12만5천원), 점심미식상(15만5천원), 저녁숙수상(22만원), 저녁미식상(24만5천원), 와인페어링(13만원), 우리술페어링(10만원)',
     'priceRange': '15~25만원대',
     'openDate': '20150701',
     'businessHour

In [ ]:
# JSON vs Dict 

dic = {'_embedded': 
            {
                'restaurants': 
                    {
                        'createdDate': 1436236247000,
                        'id': 28167,
                        'headerInfo': 
                            {
                                'nameKR': '권숙수',
                                'nameEN': 'KOREAN DINING KWONSOOKSOO',
                                'ribbonType': 'RIBBON_THREE',
                            },
                        'defaultInfo': 
                            {
                                'chefName': '권우중',
                                'website': 'http://kwonsooksoo.com',
                            }
                    }
            }
    }

jsn = '''{'_embedded': 
            {
                'restaurants': 
                    {
                        'createdDate': 1436236247000,
                        'id': 28167,
                        'headerInfo': 
                            {
                                'nameKR': '권숙수',
                                'nameEN': 'KOREAN DINING KWONSOOKSOO',
                                'ribbonType': 'RIBBON_THREE',
                            },
                        'defaultInfo': 
                            {
                                'chefName': '권우중',
                                'website': 'http://kwonsooksoo.com',
                            }
                    }
            }
    }'''


In [ ]:
dic = json.loads(jsn) # loads : json -> dic
dic

JSONDecodeError: ignored

In [ ]:
# dict로 변환 : json.loads(jsn)
dic = json.loads(jsn)
dic

# json으로 변환 : json.dumps(dict)
jsn = json.dumps(dic)
jsn

In [ ]:
# 딕셔너리 계층구조 안에서 원하는 정보 가져오기



## 1단계
dic = {'_embedded': 
            {
                'restaurants': 
                    {
                        'headerInfo': 
                            {
                                'nameKR': '권숙수',
                                'ribbonType': 'RIBBON_THREE',
                            },
                        'defaultInfo': 
                            {
                                'chefName': '권우중',
                                'website': 'http://kwonsooksoo.com',
                            }
                    }
            }
    }

print(dic['_embedded']['restaurants']['headerInfo']['nameKR'])

dic = {'_embedded': 
            {
                'restaurants': # 식당정보 여러개를 리스트로 
                    [{
                        'headerInfo': 
                            { 
                                'nameKR': '권숙수',
                                'ribbonType': 'RIBBON_THREE',
                            },
                        'defaultInfo': 
                            {
                                'chefName': '권우중',
                                'website': 'http://kwonsooksoo.com',
                            }
                    },
                     {
                        'headerInfo': 
                            { 
                                'nameKR': '가보정',
                                'ribbonType': 'RIBBON_THREE',
                            },
                        'defaultInfo': 
                            {
                                'chefName': 'OOO',
                                'website': 'http://Gabojung.com',
                            }
                    }]
            }
    }

# 리스트로 묶여있는 데까지만 연결
restaurants = dic['_embedded']['restaurants']

# 리스트는 for문으로 처리
for restaurant in restaurants:
    # for문 안에서 다시 나머지 연결
    print(restaurant['headerInfo']['nameKR'])

권숙수
가보정


### result에서 아래 항목 데이터 가져오기

In [11]:
result

# 권숙수 가져오기
# '_embedded' 'restaurants' [] 'headerInfo' 'nameKR'
# result['_embedded']['restaurants'][0]['headerInfo']['nameKR']

# 모든 레스토랑 이름 출력하기
restaurants = result['_embedded']['restaurants']

for restaurant in restaurants:
    print(restaurant['headerInfo']['nameKR'])

권숙수
낙원
다이닝마
더그린테이블
도림
도원
라망시크레
라미띠에
라연
레스쁘아뒤이부
모모야마
모수서울
무궁화
무오키
미피아체
밍글스
백리향
벽제갈비
본앤브레드
봉래헌
비스트로드욘트빌
세븐스도어
스시선수
스시인
스시조
스시효
스와니예
아리아께
알라프리마
온지음레스토랑


In [ ]:
import json 

# blue_info = {
#     '가게명' : [],
#     '리본등급' : [],
#     "사이트" : [], 
#     "전화번호": [], 
#     "휴무": [],
#     "주차":[],
#     "가격범위":[],
#     "메뉴":[],
#     "영업시간":[]
# }

names, ribbons, sites, tels, dayoffs, parks, prices, menus, times = [], [], [], [], [], [], [], [], []

In [12]:
# _embedded restaurants [] headerInfo nameKR, ribbonType
#                          defaultInfo website, phone, dayOff
#                          statusInfo parking, priceRange, menu, businessHours
                                    
# restaurants = result['_embedded']['restaurants']

# for restaurant in restaurants:
#     nameKR = restaurant['headerInfo']['nameKR']
#     ribbonType = restaurant['headerInfo']['ribbonType']
#     website = restaurant['defaultInfo']['website']
#     phone = restaurant['defaultInfo']['phone']
#     dayOff = restaurant['defaultInfo']['dayOff']
#     parking = restaurant['statusInfo']['parking']
#     priceRange = restaurant['statusInfo']['priceRange']
#     menu = restaurant['statusInfo']['menu']
#     businessHours = restaurant['statusInfo']['businessHours']


SyntaxError: ignored

In [ ]:
restaurants = result['_embedded']['restaurants']

for restaurant in restaurants:
    names.append(restaurant['headerInfo']['nameKR'])
    ribbons.append(restaurant['headerInfo']['ribbonType'])
    sites.append(restaurant['defaultInfo']['website'])
    tels.append(restaurant['defaultInfo']['phone'])
    dayoffs.append(restaurant['defaultInfo']['dayOff'])
    parks.append(restaurant['statusInfo']['parking'])
    prices.append(restaurant['statusInfo']['priceRange'])
    menus.append(restaurant['statusInfo']['menu'])
    times.append(restaurant['statusInfo']['businessHours'])


# tip1. 기록 먼저 해놓고 가져오기

# '_embedded' 'restaurants' [] 'headerInfo' 'nameKR': "권숙수"
#                                             'ribbonType' : "RIBBON_THREE"
#                             [] 'defaultInfo' 'website' : "http://kwonsooksoo.com"
#                                             'phone' : "02-542-6268 "
#                                             'dayOff' : "일, 월요일, 명절 휴무"

#                             [] 'statusInfo' 'parking' : "발레 파킹"
#                                             'priceRange' : "15~25만원대"
#                                             'menu' : 
#                                             'businessHours'

# tip2. 크롬 익스텐션 -> json formatter or jsonviewer 설치해서 브라우저에서 json 보기



In [ ]:
restaurants = result["_embedded"]["restaurants"]

for restaurant in restaurants:
    names.append(restaurant["headerInfo"]["nameKR"])
    ribbons.append(restaurant["headerInfo"]["ribbonType"])
    sites.append(restaurant["defaultInfo"]["website"])
    tels.append(restaurant["defaultInfo"]["phone"])
    dayoffs.append(restaurant["defaultInfo"]["dayOff"])
    parks.append(restaurant['statusInfo']["parking"])
    prices.append(restaurant['statusInfo']["priceRange"])
    menus.append(restaurant['statusInfo']["menu"])
    times.append(restaurant['statusInfo']["businessHours"])


blue_info = {
    '가게명' : names,
    '리본등급' : ribbons,
    "사이트" : sites, 
    "전화번호": tels, 
    "휴무": dayoffs,
    "주차": parks,
    "가격범위": prices,
    "메뉴": menus,
    "영업시간": times
}

blue_info

{'가게명': ['권숙수',
  '낙원',
  '다이닝마',
  '더그린테이블',
  '도림',
  '도원',
  '라망시크레',
  '라미띠에',
  '라연',
  '레스쁘아뒤이부',
  '모모야마',
  '모수서울',
  '무궁화',
  '무오키',
  '미피아체',
  '밍글스',
  '백리향',
  '벽제갈비',
  '본앤브레드',
  '봉래헌',
  '비스트로드욘트빌',
  '세븐스도어',
  '스시선수',
  '스시인',
  '스시조',
  '스시효',
  '스와니예',
  '아리아께',
  '알라프리마',
  '온지음레스토랑'],
 '리본등급': ['RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE',
  'RIBBON_THREE'],
 '사이트': ['http://kwonsooksoo.com',
  'http://mayfield.co.kr/2017/kor/html/restaurants/nakwon.asp',
  'http://diningma.kr',
  'http://thegreentab

In [ ]:
df_blue = pd.DataFrame(blue_info)
df_blue

,가게명,리본등급,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,권숙수,RIBBON_THREE,http://kwonsooksoo.com,02-542-6268,"일, 월요일, 명절 휴무",발레 파킹,15~25만원대,"점심숙수상(12만5천원), 점심미식상(15만5천원), 저녁숙수상(22만원), 저녁미...",12:00~15:00/18:00~22:00(마지막 주문 20:30)
1,낙원,RIBBON_THREE,http://mayfield.co.kr/2017/kor/html/restaurant...,02-2660-9010,연중무휴,가능,10~15만원대,"한우특선대갈비, 한우양념대갈비(200g 각 10만5천원), 한우꽃등심(130g 7만...","12:00~15:00/18:00~21:00│토, 일요일 12:00~21:00"
2,다이닝마,RIBBON_THREE,http://diningma.kr,02-518-7400,"일요일, 명절 당일 휴무",발레 파킹,15~25만원대,"옌바오츠코스(99만원), 만한전석코스(65만원), 다이닝마점심코스(10만원~16만원...",11:30~14:30/17:30~21:30(마지막 주문 20:30)
3,더그린테이블,RIBBON_THREE,http://thegreentable.co.kr/,02-591-2672,"월, 화요일, 명절 휴무",발레 파킹,15~25만원대,"런치코스(12만원), 디너코스(22만원), 와인페어링(14만원)",12:00~15:00/18:00~22:00 | 일요일 12:00~15:00/ 18:...
4,도림,RIBBON_THREE,http://www.lottehotel.com/seoul/ko/dining/rest...,02-317-7101,연중무휴,발레 파킹,15~25만원대,"런치코스(12만원~21만원), 디너코스(16만원~50만원), 특별주말코스(15만원~...",11:30~14:30/18:00~22:00(마지막 주문 21:00)
5,도원,RIBBON_THREE,https://www.hoteltheplaza.com/kr/dining/taoyue...,02-310-7300,연중무휴,가능,15~25만원대,"해황소스활해산물(20만원), 해황소스활바닷가재(18만원), 북경오리(16만5천원),...",11:30~14:30/18:00~22:00
6,라망시크레,RIBBON_THREE,,02-317-4003,일요일 휴무,가능,15~25만원대,"런치코스(13만원), 런치와인페어링(10만원), 디너코스(20만원), 디너와인페어링...",12:00~15:00/18:00~22:00
7,라미띠에,RIBBON_THREE,http://lamitie.modoo.at/,02-546-9621,"일요일, 명절 휴무",발레 파킹,15~25만원대,"런치코스(10만원~12만원), 디너코스(19만원~20만원)",10:00~22:00
8,라연,RIBBON_THREE,https://www.shilla.net/seoul/dining/viewDining...,02-2230-3367,연중무휴,가능,15~25만원대,"점심코스(16만원, 17만5천원), 저녁코스(20만원, 27만원)",12:00~14:30/17:30~21:30
9,레스쁘아뒤이부,RIBBON_THREE,,02-517-6034,"화요일, 명절 휴무",발레 파킹,10~15만원대,"런치코스(7만5천원), 디너코스(12만원, 14만원, 16만원), 샤퀴테리(3만원~...",12:00~15:00/18:00~22:00


In [ ]:
# 열뽑기
df_blue[['가게명', '리본등급']]

# 행뽑기
df_blue.loc[10:20]
df_blue[10:20]
# if문에서 조건이 참이라면 이라고 읽기로 한 약속이 기억나는가.
df_blue[df_blue['리본등급'] == 'RIBBON_THREE'] # '리본등급' == 'RIBBON_THREE' 이렇게 작성하는게 아니라!! 
df_blue[df_blue['주차'] == '발레 파킹']['주차'] = '가능'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,가게명,리본등급,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,권숙수,RIBBON_THREE,http://kwonsooksoo.com,02-542-6268,"일, 월요일, 명절 휴무",발레 파킹,15~25만원대,"점심숙수상(12만5천원), 점심미식상(15만5천원), 저녁숙수상(22만원), 저녁미...",12:00~15:00/18:00~22:00(마지막 주문 20:30)
1,낙원,RIBBON_THREE,http://mayfield.co.kr/2017/kor/html/restaurant...,02-2660-9010,연중무휴,가능,10~15만원대,"한우특선대갈비, 한우양념대갈비(200g 각 9만8천원), 한우꽃등심(130g 7만3...","12:00~15:00/18:00~21:30│토, 일요일 12:00~21:30"
2,더그린테이블,RIBBON_THREE,"http://thegreentable.co.kr/, https://www.insta...",02-591-2672,"월, 화요일, 명절 휴무",발레 파킹,15~25만원대,"런치코스(10만원), 디너코스(16만원)",12:00~15:00/18:00~22:00 | 일요일 12:00~15:00/ 18:...
3,도림,RIBBON_THREE,http://www.lottehotel.com/seoul/ko/dining/rest...,02-317-7101,연중무휴,발레 파킹,15~25만원대,"런치코스(4만9천원~18만원), 디너코스(10만8천원~40만원), 특별주말코스(11...",11:30~14:30/18:00~22:00
4,도원,RIBBON_THREE,https://www.hoteltheplaza.com/kr/dining/taoyue...,02-310-7300,연중무휴,가능,15~25만원대,"해황소스활해산물, 해황소스활바닷가재(각 12만원), 북경식 오리(13만원), 점심약...",11:30~14:30/18:00~22:00
5,라망시크레,RIBBON_THREE,https://www.instagram.com/lamant_secret/,02-317-4003,연중무휴,가능,15~25만원대,"런치코스(11만원), 런치와인페어링(9만원), 디너코스(18만원), 디너와인페어링(...",12:00~15:00(마지막 주문 14:00)/18:00~22:00(마지막 주문 2...
6,라미띠에,RIBBON_THREE,http://lamitie.modoo.at/,02-546-9621,"일요일, 명절 휴무",발레 파킹,15~25만원대,"런치코스(8만원~12만5천원), 디너코스(16만원~19만원)",12:00~15:00/18:00~22:30
7,라연,RIBBON_THREE,https://www.shilla.net/seoul/dining/viewDining...,02-2230-3367,연중무휴,가능,15~25만원대,"점심코스(10만9천원, 15만원, 19만9천원), 저녁코스(19만8천원, 28만원)",12:00~14:30/17:30~21:30
8,레스쁘아뒤이부,RIBBON_THREE,,02-517-6034,명절 휴무,발레 파킹,10~15만원대,"런치코스(7만5천원), 디너코스(12만원, 14만원, 16만원), 샤퀴테리(3만원~...",12:00~15:00(마지막 주문 14:00)/18:00~22:00(마지막 주문 2...
9,모모야마,RIBBON_THREE,http://www.lottehotel.com/seoul/ko/dining/dini...,02-317-7031,연중무휴,발레 파킹,15~25만원대,"모모야마코스(28만원), 생선회특선코스(24만원), 계절특선코스(21만원), 스시코...",11:30~14:30(마지막 주문 13:30)/18:00~22:00(마지막 주문 2...


In [ ]:
# name : "_embedded" "restaurants" [] "headerInfo" "nameKR"
# ribbon : "_embedded" "restaurants" [] "headerInfo" "ribbonType"
# site : "_embedded" "restaurants" [] "defaultInfo" "website"
# tel : "_embedded" "restaurants" [] "defaultInfo" "phone"
# dayoff : "_embedded" "restaurants" [] "defaultInfo" "dayOff"
# park : "_embedded" "restaurants" [] 'statusInfo' "parking"
# price : "_embedded" "restaurants" [] 'statusInfo' "priceRange"
# menu : "_embedded" "restaurants" [] 'statusInfo' "menu"
# time : "_embedded" "restaurants" [] 'statusInfo' "businessHours"

In [ ]:
# "_embedded" "restaurants": [] "headerInfo" "nameKR", "ribbonType"
# "_embedded" "restaurants": [] "defaultInfo" "website", "phone", "dayOff"
# "_embedded" "restaurants": [] "statusInfo" "parking", "visit", "menu", "priceRange", "businessHours"
# "_embedded" "restaurants": [] "juso" "roadAddrPart1"

In [ ]:
# 조건 검색
df_bluer[df_bluer['주차'] == '가능']

,가게명,리본등급,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,가보정,RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,연중무휴,가능,2~4만원대,"한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g...","11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00"
1,강천매운탕,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,연중무휴,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...",10:00~22:00
2,고기리막국수,RIBBON_TWO,http://고기리막국수.com/,031-263-1107,화요일 휴무,가능,1만원 미만,"물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원)",11:00~21:00(마지막 주문 20:30)
3,기와집순두부,RIBBON_TWO,,031-576-9009,명절 휴무,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천...",하절기 10:00~21:00 | 동절기 10:30~21:00
4,능라도,RIBBON_TWO,,031-781-3989,명절 당일 휴무,가능,1~2만원대,"평양냉면, 평양온면(각 1만4천원), 평양온반(1만2천원), 접시만두(각 1만3천원...",11:00~21:00(마지막 주문 20:30)
5,명월집,RIBBON_TWO,,031-584-4111,"월, 화, 수요일 휴무",가능,2~4만원대,돼지불고기(250g 3만원),16:00~19:30
6,사각하늘,RIBBON_TWO,,031-774-3670,"월, 화요일 휴무",가능,4~10만원대,"점심스키야키(2인 이상, 1인 4만3천원), 저녁스키야키(2인 이상, 1인 5만5천원)",12:00~22:00
7,상해루,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,월요일 휴무,가능,1~2만원대,"삼선짜장(8천5백원), 짬뽕(8천원) , 탕수육(소 2만원, 중 3만원), 오향장육...",11:00~15:00/17:00~22:00 | 주말 11:00~22:00
8,서동관,RIBBON_TWO,,031-922-7463,"화요일, 명절 휴무",가능,1~2만원대,"곰탕(1만3천원), 양곰탕(1만5천원), 특곰탕(1만7천원), 이공탕(2만원), 이...",09:00~21:00(마지막 주문 20:30)
9,세야스시,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,월요일 휴무,가능,10~20만원대,"런치오마카세(13만원), 디너오마카세(23만원), 콜키지(5만원)",12:00~15:00/18:00~22:00


In [ ]:
cols = list(df_bluer)

df_bluer[cols[:3]]

,가게명,리본등급,사이트
0,가보정,RIBBON_TWO,http://www.kabojong.co.kr
1,강천매운탕,RIBBON_TWO,http://www.365food.com/gang
2,고기리막국수,RIBBON_TWO,http://고기리막국수.com/
3,기와집순두부,RIBBON_TWO,
4,능라도,RIBBON_TWO,
5,명월집,RIBBON_TWO,
6,사각하늘,RIBBON_TWO,
7,상해루,RIBBON_TWO,http://appweb.co.kr/shanghairoo
8,서동관,RIBBON_TWO,
9,세야스시,RIBBON_TWO,http://seyasushi.modoo.at/


In [ ]:
# 딕셔너리에서 list 형변환하면 결과가 키값 리스트
# 데이터프레임을 list 형변환하면 결과가 컬럼명 리스트


,리본등급,가게명,사이트,전화번호,휴무,주차,가격범위,메뉴,영업시간
0,RIBBON_TWO,가보정,http://www.kabojong.co.kr,1600-3883,연중무휴,가능,2~4만원대,"한우생갈비(250g 7만6천원), 양념갈비(270g 5만7천원), 채끝등심(150g...","11:30~22:00 | 토, 일요일, 공휴일 11:00~22:00"
1,RIBBON_TWO,강천매운탕,http://www.365food.com/gang,031-882-5191,연중무휴,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...",10:00~22:00
2,RIBBON_TWO,고기리막국수,http://고기리막국수.com/,031-263-1107,화요일 휴무,가능,1만원 미만,"물막국수, 비빔막국수, 들기름막국수(각 9천원), 수육(소 1만4천원, 중 2만1천원)",11:00~21:00(마지막 주문 20:30)
3,RIBBON_TWO,기와집순두부,,031-576-9009,명절 휴무,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 9천원), 재래식생두부, 황태양념구이(각 1만1천...",하절기 10:00~21:00 | 동절기 10:30~21:00
4,RIBBON_TWO,능라도,,031-781-3989,명절 당일 휴무,가능,1~2만원대,"평양냉면, 평양온면(각 1만4천원), 평양온반(1만2천원), 접시만두(각 1만3천원...",11:00~21:00(마지막 주문 20:30)
5,RIBBON_TWO,명월집,,031-584-4111,"월, 화, 수요일 휴무",가능,2~4만원대,돼지불고기(250g 3만원),16:00~19:30
6,RIBBON_TWO,사각하늘,,031-774-3670,"월, 화요일 휴무",가능,4~10만원대,"점심스키야키(2인 이상, 1인 4만3천원), 저녁스키야키(2인 이상, 1인 5만5천원)",12:00~22:00
7,RIBBON_TWO,상해루,http://appweb.co.kr/shanghairoo,031-8015-0103,월요일 휴무,가능,1~2만원대,"삼선짜장(8천5백원), 짬뽕(8천원) , 탕수육(소 2만원, 중 3만원), 오향장육...",11:00~15:00/17:00~22:00 | 주말 11:00~22:00
8,RIBBON_TWO,서동관,,031-922-7463,"화요일, 명절 휴무",가능,1~2만원대,"곰탕(1만3천원), 양곰탕(1만5천원), 특곰탕(1만7천원), 이공탕(2만원), 이...",09:00~21:00(마지막 주문 20:30)
9,RIBBON_TWO,세야스시,http://seyasushi.modoo.at/,031-8015-2216,월요일 휴무,가능,10~20만원대,"런치오마카세(13만원), 디너오마카세(23만원), 콜키지(5만원)",12:00~15:00/18:00~22:00


In [ ]:
# dataframe의 컬럼 순서 조정 -> 컬럼명 리스트로 조정하는게 편하다


,가게명,영업시간,리본등급,사이트,전화번호,주차,가격범위,메뉴,휴무
0,가보정,"11:30~22:00 | 주말, 공휴일 11:00~22:00",RIBBON_TWO,http://www.kabojong.co.kr,1600-3883,가능,2~4만원대,"한우생갈비(250g 7만2천원), 양념갈비(270g 5만3천원), 채끝등심(150g...",연중무휴
1,강천매운탕,10:00~22:00,RIBBON_TWO,http://www.365food.com/gang,031-882-5191,가능,2~4만원대,"빠가사리매운탕, 잡고기매운탕(각 소 5만원, 중 6만원, 대 7만원, 특 8만원),...","첫째, 셋째 주 화요일 휴무"
2,고기리막국수,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-263-1107,가능,1만원 미만,"비빔막국수, 들기름막국수(각 8천원), 수육(소 1만3천원, 중 1만9천원)",화요일 휴무
3,기와집순두부,하절기 10:00~21:00 | 동절기 10:30~21:00,RIBBON_TWO,,031-576-9009,가능,1만원 미만,"순두부백반, 콩탕백반, 비빔밥(각 8천원), 재래식생두부, 황태양념구이(각 1만원)...",명절 휴무
4,능라도,11:00~21:00(마지막 주문 20:30),RIBBON_TWO,,031-781-3989,가능,1~2만원대,"평양냉면, 평양온면(각 1만2천원), 평양온반, 접시만두(각 1만1천원), 제육(2...",명절 당일 휴무
5,명월집,16:00~19:30,RIBBON_TWO,,031-584-4111,불가,2~4만원대,돼지불고기(250g 2만원),"월, 화, 수요일 휴무"
6,백제장,11:00~21:00,RIBBON_TWO,,031-746-4296,가능,2~4만원대,"산채정식(2인 이상, 1인 2만1천원), 숯불불고기(200g 1만8천원), 녹두빈대...",연중무휴
7,사각하늘,12:00~22:00,RIBBON_TWO,,031-774-3670,가능,4~10만원대,"점심스키야키(2인 이상, 1인 3만8천원), 저녁스키야키(2인 이상, 1인 5만5천원)","월요일, 화요일 휴무"
8,상해루,11:00~15:00/17:00~22:00 | 주말 11:00~22:00,RIBBON_TWO,http://appweb.co.kr/shanghairoo,031-8015-0103,가능,1~2만원대,"탕수육(소 1만8천원, 대 2만7천원), 멘보샤(2만7천원), 오향장육(3만3천원)...",월요일 휴무
9,세야스시,12:00~15:00/18:00~22:00,RIBBON_TWO,http://seyasushi.modoo.at/,031-8015-2216,가능,10~20만원대,"런치오마카세(10만원), 디너오마카세(20만원)",월요일 휴무
